# Barrow's Edge

How reliable is the healing provided by the Exemplar's {prd_ikons}`Barrow's Edge <2>`?
Let's test it with a {prd_weapons}`Greatpick <378>`.

In [ ]:
# Install in jupyterlite
%pip install -q pathfinder2e-stats

import xarray

import pathfinder2e_stats as pf2

In [ ]:
LEVEL = 4
HALF_HEALTH_ENEMY = False

In [ ]:
atk = (
    pf2.tables.SIMPLE_PC.weapon_attack_bonus.exemplar.sum("component")
    .sel(level=LEVEL)
    .item()
)
atk

In [ ]:
weapon_dice = pf2.tables.PC.weapon_dice.striking_rune.sel(level=LEVEL).item()

immanence = 3 if HALF_HEALTH_ENEMY else 1
weapon = pf2.armory.pathfinder.melee.greatpick(weapon_dice) + pf2.Damage(
    "spirit", 0, 0, immanence * weapon_dice
)
weapon

In [ ]:
AC = pf2.tables.SIMPLE_NPC.AC.sel(level=LEVEL)
AC.display()

Let's compare hitting an enemy without any buffs/debuffs vs. when they're flanked, grabbed, or prone:

In [ ]:
off_guard = xarray.DataArray(
    [0, -2], dims="off-guard", coords={"off-guard": [False, True]}
)

Both the challenge of the enemy and the off-guard conditions are what-if analyses:
let's roll our Strike only once and compare the output against all of them.

In [ ]:
pf2.set_config(
    check_dependent_dims=["challenge", "off-guard"],
    damage_dependent_dims=["challenge", "off-guard"],
)

Let's Strike the enemy! On a hit or critical hit, we will Spark Transcendence to heal ourselves.

In [ ]:
atk_roll = pf2.check(atk, DC=AC + off_guard)
dmg_roll = pf2.damage(atk_roll, weapon)
barrows_edge_heal = dmg_roll.total_damage // 2
pf2.outcome_counts(atk_roll).display()

## Damage distribution

In [ ]:
barrows_edge_heal.display()

In [ ]:
total_heal = barrows_edge_heal.stack(col=["challenge", "off-guard"])
means = total_heal.mean("roll").to_pandas()
stds = total_heal.std("roll").to_pandas()
_ = means.plot.barh(xerr=stds)

In [ ]:
bins = barrows_edge_heal.max().item() + 1
_ = (
    barrows_edge_heal.stack(col=["challenge", "off-guard"])
    .to_pandas()
    .hist(bins=bins, figsize=(10, 10))
)

## Conclusions
Barrow's Edge provides a solid health drip when fighting against extras, effectively allowing the Exemplar to take on an entire army and just keep going indefinitely.
However, both its reliability and mean effect drastically drop when tanking a boss - which is also what hits the hardest. So you should consider complementing it with {prd_ikons}`Scar of the Survivor <13>` or be backed by a dedicated healer.